In [1]:
import os
import tkinter.filedialog as tk
from tkinter import messagebox
from datetime import datetime
import pandas as pd
import pyodbc
import warnings
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from concurrent.futures import ThreadPoolExecutor, as_completed
from IPython.display import display
import win32com.client as client
import re
 
warnings.filterwarnings("ignore", category=UserWarning)


In [2]:
# Add your File Location's
DB_STRUCTURE_PATH = r'C:\Users\IN10031033\Desktop\Important\Querys\Jupiter NoteBook\SqlDBStructure.xlsx'
MASTER_FILE_PATH = r'C:\Users\IN10031033\Desktop\Important\Querys\Jupiter NoteBook\Mapping.xlsx'
OUTPUT_DIR = r'C:\Users\IN10031033\Desktop\Important\Final\New Folder'


In [3]:
# ---cleaning the file(do not touch)
def clean_illegal_characters(value):
    if isinstance(value, str):
        return re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\x84\x86-\x9f]', '', value)
    return value

def clean_dataframe(df):
    df_cleaned = df.copy()
    for col in df_cleaned.columns:
        if df_cleaned[col].dtype == 'object':  
            df_cleaned[col] = df_cleaned[col].apply(clean_illegal_characters)
    return df_cleaned


In [4]:
master_file = load_workbook(MASTER_FILE_PATH)
if 'LogFile' in master_file.sheetnames:
    del master_file['LogFile']
if 'Raw Data' in master_file.sheetnames:
    del master_file['Raw Data']
 
log_sheet = master_file.create_sheet('LogFile')
raw_data_sheet = master_file.create_sheet('Raw Data')


In [5]:

log_sheet.cell(row=1, column=1).value = 'Facility'
log_sheet.cell(row=1, column=2).value = 'Status'
log_sheet.cell(row=1, column=3).value = 'Row Count'
log_sheet.cell(row=1, column=4).value = 'Duration (s)'


In [6]:
# -- Add Your Sheet Name between ['%%']
db_file = load_workbook(DB_STRUCTURE_PATH)
db_sheet = db_file['All Sites']


In [7]:
# To Select Your Query
query_path = tk.askopenfilename(title="Select your SQL query", filetypes=[("SQL/Text Files", "*.sql *.txt")])
report_name = os.path.basename(query_path).replace(".txt", "").replace(".sql", "")
with open(query_path, 'r') as f:
    query = f.read()


print(f"A Query was Selected :- {report_name}")

A Query was Selected :- future followup query ( Sunil )


In [8]:
targets = []
for i in range(2, db_sheet.max_row + 1):
    facility = db_sheet.cell(row=i, column=1).value
    database = db_sheet.cell(row=i, column=2).value
    server = db_sheet.cell(row=i, column=3).value
 
    if server and database:
        targets.append((i, facility, database, server))



In [9]:
def fetch_data(row_num, facility, database, server, query):
    try:
        print(f"Connecting to {database} on {server}...")
        start = datetime.now()
        conn = pyodbc.connect(f"Driver={{SQL Server}};Server={server};Database={database};Trusted_Connection=yes;")
        df = pd.read_sql(query, conn)
        conn.close()
        duration = (datetime.now() - start).total_seconds()

        df_cleaned = clean_dataframe(df)
        
        return (row_num, facility, df_cleaned, len(df_cleaned), duration, "Success")
    except Exception as e:
        return (row_num, facility, None, 0, 0, f"Error: {str(e)}")


In [10]:
# Check for the query execution
combined_df = pd.DataFrame()
header_written = False
 
with ThreadPoolExecutor(max_workers=60) as executor:
    futures = [executor.submit(fetch_data, *args, query) for args in targets]
    for future in as_completed(futures):
        row_num, facility, df, row_count, duration, status = future.result()
 
        log_sheet.cell(row=row_num, column=1).value = facility
        log_sheet.cell(row=row_num, column=2).value = status
        log_sheet.cell(row=row_num, column=3).value = row_count
        log_sheet.cell(row=row_num, column=4).value = round(duration, 2)
 
        if df is not None and not df.empty:
            try:
                if not header_written:
                    for row in dataframe_to_rows(df, index=False, header=True):
                        raw_data_sheet.append(row)
                    header_written = True
                else:
                    for row in dataframe_to_rows(df, index=False, header=False):
                        raw_data_sheet.append(row)
                combined_df = pd.concat([combined_df, df], ignore_index=True)
            except Exception as e:
                print(f"Error writing data for {facility}: {e}")
                log_sheet.cell(row=row_num, column=2).value = f"Write Error: {str(e)}"


Connecting to TranA116 on AHS-A2RSAS03.extapp.local...
Connecting to TranA118 on AHS-A2RSAS03.extapp.local...
Connecting to TranA120 on AHS-A2RSAS03.extapp.local...
Connecting to TranA122 on AHS-A2RSAS03.extapp.local...
Connecting to TranA124 on AHS-A2RSAS03.extapp.local...
Connecting to TranA125 on AHS-A2RSAS03.extapp.local...
Connecting to TranA126 on AHS-A2RSAS03.extapp.local...
Connecting to TranA127 on AHS-A2RSAS03.extapp.local...
Connecting to TranA128 on AHS-A2RSAS03.extapp.local...
Connecting to TranA129 on AHS-A2RSAS03.extapp.local...
Connecting to TranA130 on AHS-A2RSAS03.extapp.local...
Connecting to TranA132 on AHS-A2RSAS03.extapp.local...
Connecting to TranA134 on AHS-A2RSAS03.extapp.local...
Connecting to TranA138 on AHS-A2RSAS03.extapp.local...
Connecting to TranA139 on AHS-A2RSAS03.extapp.local...
Connecting to TranA140 on AHS-A2RSAS03.extapp.local...
Connecting to TranA141 on AHS-A2RSAS03.extapp.local...
Connecting to TranA142 on AHS-A2RSAS03.extapp.local...
Connecting

Connecting to TranP116 on AHS-TRAN15.accretivehealth.local...
Connecting to TranSAWI on AHS-TRAN15.accretivehealth.local...
Connecting to TranHMPA on AHS-Tran16.accretivehealth.local...
Connecting to TranJAKS on AHS-TRAN16.accretivehealth.local...
Connecting to TranJBKS on AHS-TRAN16.accretivehealth.local...
Connecting to TranMHKS on AHS-TRAN16.accretivehealth.local...
Connecting to TranRHKS on AHS-TRAN16.accretivehealth.local...
Connecting to TranRPTN on AHS-TRAN16.accretivehealth.local...
Connecting to TranSMPA on AHS-Tran16.accretivehealth.local...
Connecting to TranSTKS on AHS-TRAN16.accretivehealth.local...
Connecting to TranVFKS on AHS-TRAN16.accretivehealth.local...
Connecting to TranVJKS on AHS-TRAN16.accretivehealth.local...
Connecting to TranVPKS on AHS-TRAN16.accretivehealth.local...
Connecting to TranVWKS on AHS-TRAN16.accretivehealth.local...
Connecting to TranWHKS on AHS-TRAN16.accretivehealth.local...
Connecting to TranBMAL on AHS-TRAN17.accretivehealth.local...
Connecti

Connecting to TranPSIL on ALPTRAN29.accretivehealth.local...
Connecting to TranSWIN on ALPTRAN29.accretivehealth.local...
Connecting to TranVEIN on ALPTRAN29.accretivehealth.local...
Connecting to TranJCIL on ALPTRAN30.accretivehealth.local...
Connecting to TranMIWI on ALPTRAN30.accretivehealth.local...
Connecting to TranMPWI on ALPTRAN30.accretivehealth.local...
Connecting to TranPFIL on ALPTRAN30.accretivehealth.local...
Connecting to TranSAMD on ALPTRAN30.accretivehealth.local...
Connecting to TranSBWI on ALPTRAN30.accretivehealth.local...
Connecting to TranSCWI on ALPTRAN30.accretivehealth.local...
Connecting to TranSEWI on ALPTRAN30.accretivehealth.local...
Connecting to TranUCAL on ALPTRAN30.accretivehealth.local...
Connecting to TranULAL on ALPTRAN30.accretivehealth.local...
Connecting to TranUMAL on ALPTRAN30.accretivehealth.local...
Connecting to TranUSAL on ALPTRAN30.accretivehealth.local...
Connecting to TranAARX on ALPTRAN31.accretivehealth.local...
Connecting to TranBSTX o

In [11]:
print("Inserting 'Site' column with VLOOKUP formula...")
raw_data_sheet.insert_cols(2)
raw_data_sheet.cell(row=1, column=2).value = 'Site'
for row in range(2, raw_data_sheet.max_row + 1):
    formula = f'=VLOOKUP(A{row},Mapp!A:B,2,0)'
    raw_data_sheet.cell(row=row, column=2).value = formula


Inserting 'Site' column with VLOOKUP formula...


ValueError: Row numbers must be between 1 and 1048576

In [15]:
# Code Completed File Saved
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
output_file = os.path.join(OUTPUT_DIR, f"{report_name}_{timestamp}.xlsx")
master_file.save(output_file)
print(f"✅ File saved: {output_file}")


✅ File saved: C:\Users\IN10031033\Desktop\Important\Final\New Folder\future followup query ( Sunil )_20250911-201402.xlsx


In [ ]:
# Validation 
display(combined_df.head())
print(f"Combined dataframe shape: {combined_df.shape}")

In [ ]:
# Pop-UP
import winsound
import ctypes
winsound.MessageBeep()
ctypes.windll.user32.MessageBoxW(0, "Hey your Code is completed", "Task Complete", 0x40000)

In [ ]:
def send_email(file_path, report_name):
    """Send the report via email using Outlook."""
    print("Sending email...")
    try:
        outlook = client.Dispatch('Outlook.Application')
        message = outlook.CreateItem(0)
        message.to = "RGUPTA20@R1RCM.COM"
        message.subject = report_name
        message.body = (
            f"Hi All,\n\n"
            f"Please find the attached report for.."
        )
        message.Attachments.Add(file_path)
        message.Send()
        print("Email sent successfully.")
    except Exception as e:
        print(f"Error sending email: {e}")

output_file_path = output_file 
#send_email(output_file_path, report_name)


In [ ]:
786.59+91.50